In [1]:
import pandas as pd
import numpy as np
import time
from keras.models import Sequential
from keras.layers import Dense
from sklearn import preprocessing

Using TensorFlow backend.


In [2]:
def expand_column(df, test, column):
    #print(column + " - " + str(len(df[column].unique())))
    for value in df[column].unique():
        if (str(value) != 'nan'):
            #print(column + " -  " + str(value))
            new_column = column + '_' + str(value)
            df.loc[df[column] != value, new_column] = 0
            df.loc[df[column] == value, new_column] = 1
            test.loc[test[column] != value, new_column] = 0
            test.loc[test[column] == value, new_column] = 1

def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [3]:
df = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

enum_columns = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 
                'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 
                'OverallCond', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 
                'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 
                'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'Fireplaces', 
                'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 
                'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']

for col in enum_columns:
    expand_column(df, test, col)

df.drop(enum_columns, axis=1, inplace=True)
test.drop(enum_columns, axis=1, inplace=True)

df.fillna(df.mean(), inplace=True)
test.fillna(df.mean(), inplace=True)

df.drop('Id', axis=1, inplace=True)
textIds = test.loc[:,'Id']
test.drop('Id', axis=1, inplace=True)

Y = df.loc[:,'SalePrice']

X = df.drop('SalePrice', axis=1) # , inplace=True
X = normalize(X) # .iloc[:,0:8]

In [4]:
print(X.shape)
batch_size = X.shape[0]
input_dim = X.shape[1]
print(input_dim)

(1460, 322)
322


In [5]:
X

,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,SaleType_CWD,SaleType_ConLw,SaleType_Con,SaleType_Oth,SaleCondition_Normal,SaleCondition_Abnorml,SaleCondition_Partial,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family
0,0.150685,0.033420,0.949275,0.883333,0.122500,0.125089,0.000000,0.064212,0.140098,0.119780,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.202055,0.038795,0.753623,0.433333,0.000000,0.173281,0.000000,0.121575,0.206547,0.212942,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.160959,0.046507,0.934783,0.866667,0.101250,0.086109,0.000000,0.185788,0.150573,0.134465,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.133562,0.038561,0.311594,0.333333,0.000000,0.038271,0.000000,0.231164,0.123732,0.143873,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.215753,0.060576,0.927536,0.833333,0.218750,0.116052,0.000000,0.209760,0.187398,0.186095,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,0.219178,0.059899,0.876812,0.750000,0.000000,0.129695,0.000000,0.027397,0.130278,0.106012,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,0.184932,0.041057,0.956522,0.916667,0.116250,0.242558,0.000000,0.135702,0.275941,0.312070,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,0.167979,0.042450,0.731884,0.383333,0.150000,0.152197,0.021710,0.092466,0.181178,0.177375,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,0.102740,0.022529,0.427536,0.000000,0.000000,0.000000,0.000000,0.407534,0.155810,0.157871,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,0.099315,0.028605,0.485507,0.000000,0.000000,0.150780,0.000000,0.059932,0.162193,0.170491,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [6]:
import tensorflow as tf

class NBatchLogger(tf.keras.callbacks.Callback):
    """
    A Logger that log average performance per `display` steps.
    """
    def __init__(self, display):
        self.step = 0
        self.display = display
        self.metric_cache = {}

    def on_batch_end(self, batch, logs={}):
        self.step += 1
        for k in self.params['metrics']:
            if k in logs:
                self.metric_cache[k] = self.metric_cache.get(k, 0) + logs[k]
        if self.step % self.display == 0:
            metrics_log = ''
            for (k, v) in self.metric_cache.items():
                val = v / self.display
                #if abs(val) > 1e-3:
                metrics_log += ' - %s: %.4f' % (k, val)
                #else:
                #    metrics_log += ' - %s: %.4e' % (k, val)
            print('step: {}/{} ... {}'.format(self.step, self.params['steps'], metrics_log))
            self.metric_cache.clear()

In [7]:
# print(X.isnull().any())
out_batch = NBatchLogger(display=10000)

from keras.callbacks import CSVLogger

ts = int(time.time())
csv_logger = CSVLogger('log_' + str(ts) + '.csv', append=True, separator='\t')

# create model
model = Sequential()
model.add(Dense(input_dim, activation='relu', input_dim=input_dim))
model.add(Dense(input_dim, activation='relu')) 
model.add(Dense(input_dim, activation='relu')) 
model.add(Dense(int(input_dim / 2), activation='relu')) #
model.add(Dense(1)) 

#, activation='sigmoid'#  relu
# Compile model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X, Y, epochs=300000, batch_size=batch_size, verbose=0, callbacks=[csv_logger]) # , callbacks=[out_batch]

# evaluate the model
scores = model.evaluate(X, Y)

print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

1460/1460 [==============================] - 0s 30us/step

acc: 100.00%


In [8]:
test

,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,SaleType_CWD,SaleType_ConLw,SaleType_Con,SaleType_Oth,SaleCondition_Normal,SaleCondition_Abnorml,SaleCondition_Partial,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family
0,80.000000,11622,1961,1961,0.0,468.0,144.0,270.0,882.0,896,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,81.000000,14267,1958,1958,108.0,923.0,0.0,406.0,1329.0,1329,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,74.000000,13830,1997,1998,0.0,791.0,0.0,137.0,928.0,928,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,78.000000,9978,1998,1998,20.0,602.0,0.0,324.0,926.0,926,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,43.000000,5005,1992,1992,0.0,263.0,0.0,1017.0,1280.0,1280,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,75.000000,10000,1993,1994,0.0,0.0,0.0,763.0,763.0,763,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,70.049958,7980,1992,2007,0.0,935.0,0.0,233.0,1168.0,1187,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,63.000000,8402,1998,1998,0.0,0.0,0.0,789.0,789.0,789,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,85.000000,10176,1990,1990,0.0,637.0,0.0,663.0,1300.0,1341,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,70.000000,8400,1970,1970,0.0,804.0,78.0,0.0,882.0,882,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [9]:
testX = normalize(test) # .iloc[:,0:8]
print(testX.shape)

(1459, 322)


In [14]:
testX

,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,SaleType_CWD,SaleType_ConLw,SaleType_Con,SaleType_Oth,SaleCondition_Normal,SaleCondition_Abnorml,SaleCondition_Partial,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family
0,0.329609,0.184147,0.625954,0.183333,0.000000,0.116708,0.094364,0.126168,0.173111,0.104309,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.335196,0.232124,0.603053,0.133333,0.083721,0.230175,0.000000,0.189720,0.260844,0.196672,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.296089,0.224197,0.900763,0.800000,0.000000,0.197257,0.000000,0.064019,0.182139,0.111135,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.318436,0.154326,0.908397,0.800000,0.015504,0.150125,0.000000,0.151402,0.181747,0.110708,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.122905,0.064121,0.862595,0.700000,0.000000,0.065586,0.000000,0.475234,0.251227,0.186220,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,0.301676,0.154725,0.870229,0.733333,0.000000,0.000000,0.000000,0.356542,0.149755,0.075939,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,0.274022,0.118085,0.862595,0.950000,0.000000,0.233167,0.000000,0.108879,0.229244,0.166382,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,0.234637,0.125739,0.908397,0.800000,0.000000,0.000000,0.000000,0.368692,0.154858,0.081485,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,0.357542,0.157918,0.847328,0.666667,0.000000,0.158853,0.000000,0.309813,0.255152,0.199232,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,0.273743,0.125703,0.694656,0.333333,0.000000,0.200499,0.051114,0.000000,0.173111,0.101323,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [74]:
testX.isnull().any()

LotFrontage              False
LotArea                  False
YearBuilt                False
YearRemodAdd             False
MasVnrArea               False
BsmtFinSF1               False
BsmtFinSF2               False
BsmtUnfSF                False
TotalBsmtSF              False
1stFlrSF                 False
2ndFlrSF                 False
LowQualFinSF             False
GrLivArea                False
BsmtFullBath             False
BsmtHalfBath             False
FullBath                 False
HalfBath                 False
BedroomAbvGr             False
KitchenAbvGr             False
TotRmsAbvGrd             False
GarageYrBlt              False
GarageCars               False
GarageArea               False
WoodDeckSF               False
OpenPorchSF              False
EnclosedPorch            False
3SsnPorch                False
ScreenPorch              False
PoolArea                 False
MiscVal                  False
                         ...  
GarageCond_Ex            False
PavedDri

In [28]:
#testX.fillna(testX.mean(), inplace=True)

In [37]:
testX['MiscFeature_TenC'] = 0.0
testX['PoolQC_Fa'] = 0.0
testX['GarageQual_Ex'] = 0.0
testX['Electrical_Mix'] = 0.0
testX['Heating_OthW'] = 0.0
testX['Heating_Floor'] = 0.0
testX['Exterior2nd_Other'] = 0.0
testX['Exterior1st_Stone'] = 0.0
testX['Exterior1st_ImStucc'] = 0.0
testX['RoofMatl_Roll'] = 0.0
testX['RoofMatl_ClyTile'] = 0.0
testX['RoofMatl_Membran'] = 0.0
testX['RoofMatl_Metal'] = 0.0
testX['HouseStyle_2.5Fin'] = 0.0
testX['Condition2_RRNn'] = 0.0
testX['Condition2_RRAn'] = 0.0
testX['Condition2_RRAe'] = 0.0
testX['Utilities_NoSeWa'] = 0.0

In [39]:
# Fit the model
testY = model.predict(testX, batch_size=100, verbose=2)

array([[134451.02],
       [169382.05],
       [182738.11],
       ...,
       [137273.61],
       [107683.55],
       [208541.56]], dtype=float32)

In [71]:
# testY.T[0]
#np.array(textIds[0:1459])

array([1461, 1462, 1463, ..., 2917, 2918, 2919], dtype=object)

In [73]:
result = {'Id': np.array(textIds[0:1459]), 'SalePrice': testY.T[0]}
testResult = pd.DataFrame(data=result)

#textIds['SalePrice'] = testY

testResult.to_csv(r'./data/result.csv', index=None, sep=',', mode='a')